In [22]:
import tensorflow as tf
import numpy as np
tf.__version__

'2.18.0'

In [37]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, GRU, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer


In [24]:
mark_start = 'ssss ' #decoderın çıktı üretilmesi için start token gerekiyor
mark_end = ' eeee'

In [25]:
data_src = []
data_dest = []

In [26]:
for line in open('tur.txt', encoding='UTF-8'):
    en_text, tr_text = line.rstrip().split('\t')

    tr_text = mark_start + tr_text + mark_end           #tur.text veri setindeki ingilizce ve türkçe veriler ayrıştırılır ve turkçe verilerin başına ve sonuna end start markları eklenir.

    data_src.append(en_text)
    data_dest.append(tr_text)

In [27]:
data_src[100]  #ayrıştırma işleminde sorun olup olmadığı kontrol edilir.

'I drove.'

In [28]:
data_dest[100]

'ssss Araba sürdüm. eeee'

In [29]:
data_src[200000]

'Can you see anything missing?'

In [30]:
data_dest[200000]

'ssss Eksik bir şey görebiliyor musun? eeee'

In [31]:
len(data_src) #ingilizce verilerin uzunluğu kontrol edilir.

473035

In [32]:
class TokenizerWrap(Tokenizer):  #Daha sonra yapılacak olan kullanımlarada kolaylık sağlanması amacıyla sınıflandırma kullanılmıştır.
    def __init__(self, texts, padding, reverse=False, num_words=None):  #constructer kullanılarak tanımlanan her sınıf değeri için bir alt fonksiyon ile belirlenen parametrelerin uygun olması koşuluyla tokenizer işlemi gerçekleşir.
        Tokenizer.__init__(self, num_words=num_words)

        self.fit_on_texts(texts)

        self.index_to_word = dict(zip(self.word_index.values(), self.word_index.keys()))

        self.tokens = self.texts_to_sequences(texts)

        if reverse:
            self.tokens = [list(reversed(x)) for x in self.tokens] #padding işlemi uygulanır. Padding işlemi uygulanırken vectör oluşturulma durumunda cümlelerin ters dönme durumları göz önüne alınarak post ve pre olarak uygulanır.
            truncating = 'pre'
        else:
            truncating = 'post'

        self.num_tokens = [len(x) for x in self.tokens]
        self.max_tokens = np.mean(self.num_tokens) + 2 * np.std(self.num_tokens) #İşlenicek cümleler uzunluklarının aynı olması gerektiği için tüm cümleleerin ortalama uzunlarına göre ideal uzunluk değeri hesaplanarak tüm cümleler o uzunluğa uygun olarak kesilir yada boş dolarak doldurulur.
        self.max_tokens = int(self.max_tokens)

        self.tokens_padded = pad_sequences(self.tokens,
                                           maxlen=self.max_tokens,
                                           padding=padding,
                                           truncating=truncating)

    def token_to_word(self, token):
        word = ' ' if token == 0 else self.index_to_word[token]  #Decoderdan alınan tokenlerin kelimelere dönüştürme işleminin yapılması için uygun fonksiyon hazırlanır
        return word

    def tokens_to_string(self, tokens):
        words = [self.index_to_word[token] for token in tokens if token != 0]
        text = ' '.join(words)
        return text

    def text_to_tokens(self, text, padding, reverse=False):
        tokens = self.texts_to_sequences([text])
        tokens = np.array(tokens)

        if reverse:                                             #Üst tarafta hazırlanan padding parametrelerinde olduğu gibi uygun post veya pre işlemine ve uzunluk belirleme işlemine göre uygun tokenizer işlemi yapılır.
            tokens = np.flip(tokens, axis=1)
            truncating = 'pre'
        else:
            truncating = 'post'

        tokens = pad_sequences(tokens,
                               maxlen=self.max_tokens,
                               padding=padding,
                               truncating=truncating)

        return tokens

In [36]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer_src = TokenizerWrap(texts=data_src,  #Tokenleştirme işlemi yapılır.
                              padding='pre',
                              reverse=True,
                              num_words=None)

In [38]:
tokenizer_dest = TokenizerWrap(texts=data_dest,
                              padding='post',
                              reverse=False,
                              num_words=None)

In [39]:
tokens_src = tokenizer_src.tokens_padded
tokens_dest = tokenizer_dest.tokens_padded
print(tokens_src.shape)
print(tokens_dest.shape)

(473035, 11)
(473035, 10)


In [40]:
tokens_dest[200000] #Tokenleştirilme işlemi kontrol edilir.

array([   1, 2391,    4,   18, 4127,   48,    2,    0,    0,    0],
      dtype=int32)

In [41]:
tokenizer_dest.tokens_to_string(tokens_dest[200000])

'ssss eksik bir şey görebiliyor musun eeee'

In [42]:
tokens_src[200000]

array([   0,    0,    0,    0,    0,    0, 1028,  113,   95,    5,   39],
      dtype=int32)

In [43]:
tokenizer_src.tokens_to_string(tokens_src[200000])

'missing anything see you can'

In [44]:
data_src[200000]

'Can you see anything missing?'

In [45]:
token_start = tokenizer_dest.word_index[mark_start.strip()]
token_start   #Start ve end tokenleri kullanım sıkılığından dolayı 1 ve 2 değerlerini alır.

1

In [46]:
token_end = tokenizer_dest.word_index[mark_end.strip()]
token_end

2

In [47]:
encoder_input_data = tokens_src #encoder input data tanımlanır.

In [48]:
decoder_input_data = tokens_dest[:, :-1]
decoder_output_data = tokens_dest[:, 1:]

In [49]:
encoder_input_data[200000]

array([   0,    0,    0,    0,    0,    0, 1028,  113,   95,    5,   39],
      dtype=int32)

In [50]:
decoder_input_data[200000]

array([   1, 2391,    4,   18, 4127,   48,    2,    0,    0], dtype=int32)

In [51]:
decoder_output_data[200000]

array([2391,    4,   18, 4127,   48,    2,    0,    0,    0], dtype=int32)

In [52]:
tokenizer_dest.tokens_to_string(decoder_input_data[200000])

'ssss eksik bir şey görebiliyor musun eeee'

In [53]:
tokenizer_dest.tokens_to_string(decoder_output_data[200000])

'eksik bir şey görebiliyor musun eeee'

In [54]:
num_encoder_words = len(tokenizer_src.word_index)
num_decoder_words = len(tokenizer_dest.word_index)

In [55]:
num_encoder_words

21315

In [56]:
num_decoder_words

94058

In [57]:
embedding_size = 100

In [58]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [59]:
word2vec = {}
with open('/content/drive/MyDrive/glove.6B.100d.txt', encoding='UTF-8') as f: #Daha önceden hazırlanmış glove vectörüden alınan vectörler ve kelimeler  ayrılarak bir sözlük içinde saklanır.
    for line in f:
        values = line.split()
        word = values[0]
        vec = np.asarray(values[1:], dtype='float32')
        word2vec[word] = vec

In [60]:
embedding_matrix = np.random.uniform(-1, 1, (num_encoder_words, embedding_size)) #Alınan token değerlerini vectöre dönüştürmek ve bu vectörlere göre layerları eğitmek için araya bir embedding  layer eklememiz gerekiyor.
for word, i in tokenizer_src.word_index.items():
    if i < num_encoder_words:
        embedding_vector = word2vec.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [61]:
embedding_matrix.shape

(21315, 100)

In [62]:
encoder_input = Input(shape=(None,), name='encoder_input')

In [63]:
encoder_embedding = Embedding(input_dim=num_encoder_words, #Oluşturulan bu layerın encodera gelen kelimeleri alması ve 100 vectör uzunluğuna göre çevirmesi gerekiyor.
                              output_dim=embedding_size,
                              weights=[embedding_matrix],
                              trainable=True,
                              name='encoder_embedding')

In [64]:
state_size = 256

In [65]:
encoder_gru1 = GRU(state_size, name='encoder_gru1', return_sequences=True)
encoder_gru2 = GRU(state_size, name='encoder_gru2', return_sequences=True)  #Son layer  çıktı verdiği için son layer hariç diğer layerların eğitilebilr olması gerekiyor
encoder_gru3 = GRU(state_size, name='encoder_gru3', return_sequences=False)

In [66]:
def connect_encoder():
    net = encoder_input

    net = encoder_embedding(net)  #Oluşturulan embedding ve encoder layerlarını birbirine bağlalıyoruz.

    net = encoder_gru1(net)
    net = encoder_gru2(net)
    net = encoder_gru3(net)

    encoder_output = net

    return encoder_output

In [67]:
encoder_output = connect_encoder()

In [68]:
decoder_initial_state = Input(shape=(state_size,), name='decoder_initial_state')

In [69]:
decoder_input = Input(shape=(None,), name='decoder_input')

In [70]:
decoder_embedding = Embedding(input_dim=num_decoder_words,
                              output_dim=embedding_size,
                              name='decoder_embedding')

In [71]:
decoder_gru1 = GRU(state_size, name='decoder_gru1', return_sequences=True)
decoder_gru2 = GRU(state_size, name='decoder_gru2', return_sequences=True)  #Son layerdan sonra bir dens edaha ekleyeceğimiz için hepsi eğitilebilir olabilir.
decoder_gru3 = GRU(state_size, name='decoder_gru3', return_sequences=True)

In [72]:
decoder_dense = Dense(num_decoder_words,
                      activation='linear', #En son katmana bir lineer layer ekleyerek çıktı alıyoruz.
                      name='decoder_output')

In [73]:
def connect_decoder(initial_state):
    net = decoder_input

    net = decoder_embedding(net)

    net = decoder_gru1(net, initial_state=initial_state)
    net = decoder_gru2(net, initial_state=initial_state)
    net = decoder_gru3(net, initial_state=initial_state)

    decoder_output = decoder_dense(net)

    return decoder_output

In [74]:
decoder_output = connect_decoder(initial_state=encoder_output)

model_train = Model(inputs=[encoder_input, decoder_input], outputs=[decoder_output]) #Modellerin eğitim sırasında uygun sırada verileri alıp eğitilmesi için modellerin çıktılarının ve girdilerinin birbirine bağlı olması gerekiyor.

In [75]:
model_encoder = Model(inputs=[encoder_input], outputs=[encoder_output])

In [76]:
decoder_output = connect_decoder(initial_state=decoder_initial_state)

model_decoder = Model(inputs=[decoder_input, decoder_initial_state], outputs=[decoder_output])

In [92]:
def sparse_cross_entropy(y_true, y_pred):
    # y_true'yu int32’ye çeviriyoruz
    y_true = tf.cast(y_true, tf.int32)
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
        labels=y_true,
        logits=y_pred
    )
    return tf.reduce_mean(loss)


In [93]:
from tensorflow.keras.optimizers import RMSprop

optimizer = RMSprop(learning_rate=1e-3)
 #Uygun optimizer değeri verildi.

In [94]:
from tensorflow.keras.layers import Input

decoder_target = Input(shape=(None,), dtype='int32', name='decoder_target')


model_train.compile(optimizer=optimizer,
                    loss=sparse_cross_entropy,
                    target_tensors=[decoder_target])

In [95]:
path_checkpoint = 'checkpoint.weights.h5'   #Daha sonradan kullanım kolaylığı olması açısından kayıt işlemi eklendi.
checkpoint = ModelCheckpoint(filepath=path_checkpoint, save_weights_only=True)

In [96]:
try:
    model_train.load_weights(path_checkpoint)
except Exception as error:
    print('Checkpoint yüklenirken hata oluştu. Eğitime sıfırdan başlanıyor.')
    print(error)

Checkpoint yüklenirken hata oluştu. Eğitime sıfırdan başlanıyor.
[Errno 2] Unable to synchronously open file (unable to open file: name = 'checkpoint.weights.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)


In [97]:
x_data = {'encoder_input': encoder_input_data, 'decoder_input': decoder_input_data} #İnputlar ve outputlar belirlendi.

In [98]:
y_data = {'decoder_output': decoder_output_data}

In [100]:
# 1) Modeli derleyin (bir kez, fit()'ten önce)
model_train.compile(
    optimizer=optimizer,
    loss=sparse_cross_entropy
)

# 2) Girdi ve çıktı dizilerini kesinlikle integer tipine dönüştürün
encoder_input_data = encoder_input_data.astype('int32')
decoder_input_data = decoder_input_data.astype('int32')
decoder_output_data = decoder_output_data.astype('int32')

# 3) Eğitim çağrısını şu şekilde yapın (dict yerine liste+array kullanıyoruz)
model_train.fit(
    x=[encoder_input_data, decoder_input_data],
    y=decoder_output_data,
    batch_size=64,
    epochs=2,
    callbacks=[checkpoint]
)


Epoch 1/2
7392/7392 ━━━━━━━━━━━━━━━━━━━━ 1640s 222ms/step - loss: 4.0181
Epoch 2/2
7392/7392 ━━━━━━━━━━━━━━━━━━━━ 1635s 221ms/step - loss: 2.6246


In [103]:
def translate(input_text, true_output_text=None):
    # 1) Girdi cümleyi token’lara çevir
    input_tokens = tokenizer_src.text_to_tokens(
        text=input_text,
        reverse=True,
        padding='pre'
    )

    # 2) Encoder’dan state alırken liste içinde veriyoruz
    initial_state = model_encoder.predict([input_tokens])

    max_tokens = tokenizer_dest.max_tokens
    # 3) np.int kaldırıldı, yerine Python int kullanıldı
    decoder_input_data = np.zeros(shape=(1, max_tokens), dtype=int)

    token_int = token_start
    output_text = ''
    count_tokens = 0

    while token_int != token_end and count_tokens < max_tokens:
        decoder_input_data[0, count_tokens] = token_int

        # 4) Decoder’a da liste halinde veriyoruz
        decoder_output = model_decoder.predict([decoder_input_data, initial_state])

        token_onehot = decoder_output[0, count_tokens, :]
        token_int = np.argmax(token_onehot)

        sampled_word = tokenizer_dest.token_to_word(token_int)
        output_text += ' ' + sampled_word
        count_tokens += 1

    print('Input text:')
    print(input_text, '\n')

    print('Translated text:')
    print(output_text, '\n')

    if true_output_text is not None:
        print('True output text:')
        print(true_output_text, '\n')


In [106]:
translate(input_text=data_src[300000], true_output_text=data_dest[300000]) #Çıktılar orjinal değerlere göre kontrol edilir.

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
Input text:
Tom will certainly try to do that. 

Translated text:
 tom kesinlikle bunu yapmak zorunda eeee 

True output text:
ssss Tom kesinlikle onu yapmaya çalışacak. eeee 



In [115]:
translate(input_text='What do you know about ?')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Input text:
What do you know about ? 

Translated text:
 ne hakkında biliyorsun eeee 



In [110]:
# H5 formatında:
model_train.save('seq2seq_model.h5')

